In [ ]:
import cv2
import time


plate_cas = cv2.CascadeClassifier('indian_license_plate.xml')


#To detect license plates of vehicle out of the frame , if any
def detect(img):
    plate_img = img.copy()
    
    plate_rect = np.array([])
    i=1
    while plate_rect.shape != (1,4):
        
        i+=1
        plate_rect = plate_cas.detectMultiScale(plate_img,scaleFactor=1.3,
                                               minNeighbors = i)
        plate_rect = np.array(plate_rect)
        if i==21:
            break
        print(plate_rect)

    p=plate_rect[0][0]
    q=plate_rect[0][1]
    r=plate_rect[0][2]
    s=plate_rect[0][3]

    for (x,y,w,h) in plate_rect:
        cv2.rectangle(plate_img,(x,y),(x+w,y+h),(255,0,0),4)

    return plate_img[q:q+s,p:p+r,:]

#To process and extract the license plate number from the cropped image of license plate generated by detect() function
def process(img):
    
    #Pre-process 1
    
    gray_img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY) # Converting to grayscale for preprocessing
    
    #Pre-process 2
    
    gray_img = cv2.GaussianBlur(gray_img,(5,5),10)  # applying gaussian blur
    
    #Pre-process 3
    
    gray_img = cv2.morphologyEx(gray_img,cv2.MORPH_OPEN,kernel) #Morphologigal filter
    
    #Pre-process 4
    
    _,th1 = cv2.threshold(gray_result, 180, 255, cv2.THRESH_BINARY)  # applying apt thresholding
    
    num1 = pytesseract.image_to_string(th1) # thresholded image
    num2 = pytesseract.image_to_string(gray_img) #non - thresholded image
    
    
    
    return img , num1 if len(num1) > len(num2) else num2



cap = cv2.VideoCapture('')

if cap.isOpened() == False:
    print('ERROR : file not found! OR Wrong Codec!')
    
while cap.isOpened():
    
    ret,frame = cap.read()
    
    pl = detect(frame)
    thresholded_plate, num = process(pl)
    
    if num != '':
        cv2.imwrite("Plate_DIR\\new_plate", thresholded_plate)
        with open('Plates_directory.txt','a') as f:
            f.write('\n{}'.format(num))

        
    if ret == True:
        
        # WRITER 20 FPS
        time.sleep(1/20)
        cv2.imshow('frame',frame)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    else:
        break
        
cap.release()
cv2.destroyAllWindows()


